<a href="https://colab.research.google.com/github/ManjuRama/FinMath/blob/main/NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_percentage_error
import matplotlib.pyplot as plt

# Example DataFrame: Ensure your real dataset is properly loaded
# df = pd.read_csv('your_dataset.csv')

# Generate synthetic data for demonstration purposes
np.random.seed(42)
minutes_per_hour = 60
hours = 9
n = minutes_per_hour * hours  # 9 hours of minute-by-minute data (60 * 9 = 540 minutes)
data = {
    'volumelastminute': np.random.rand(n) * 100,
    'volumelasthour': np.random.rand(n) * 500,
    'spread': np.random.rand(n) * 5,
    'midprice': np.random.rand(n) * 1000,
    'nexthourvolume': np.random.rand(n) * 600
}
df = pd.DataFrame(data)

# Hyperparameters
window_size = 60  # Rolling window size
input_size = 4    # Number of input features (volumelastminute, volumelasthour, spread, midprice)
hidden_size = 64  # Number of hidden units
output_size = 1   # Predicting one output (nexthourvolume)
learning_rate = 0.001
epochs = 20

# Neural network definition
class VolumePredictorNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(VolumePredictorNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Prepare input features and target using a rolling window
def create_rolling_window_data(df, window_size):
    X = []
    y = []
    for i in range(window_size, len(df) - window_size):
        # Extract past 60 minutes of features
        X_window = df[['volumelastminute', 'volumelasthour', 'spread', 'midprice']].iloc[i-window_size:i].values
        # Average target volume for the next hour
        y_window = df['nexthourvolume'].iloc[i:i+window_size].mean()
        X.append(X_window)
        y.append(y_window)

    return np.array(X), np.array(y)

# Generate rolling window data
X, y = create_rolling_window_data(df, window_size)

# Convert the data to torch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32).view(-1, 1)

# Split data into training and test sets (80% train, 20% test)
train_size = int(0.8 * len(X_tensor))
X_train, X_test = X_tensor[:train_size], X_tensor[train_size:]
y_train, y_test = y_tensor[:train_size], y_tensor[train_size:]

# Initialize the model, loss function, and optimizer
model = VolumePredictorNN(input_size, hidden_size, output_size)
criterion = nn.L1Loss()  # We'll use MAE and calculate MAPE manually
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()

    # Zero the gradients
    optimizer.zero_grad()

    # Forward pass
    outputs = model(X_train.mean(dim=1))  # Averaging the past 60 minutes for each window
    loss = criterion(outputs, y_train)

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    # Print loss
    if (epoch+1) % 5 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

# Testing the model
model.eval()
with torch.no_grad():
    predictions = model(X_test.mean(dim=1))
    test_loss = criterion(predictions, y_test).item()

# Calculate MAPE for test set
y_test_np = y_test.numpy()
predictions_np = predictions.numpy()
mape = mean_absolute_percentage_error(y_test_np, predictions_np)
print(f"Test MAPE: {mape:.4f}")

# Plot actual vs predicted values for the test set
plt.figure(figsize=(10, 6))
plt.plot(y_test_np, label='Actual nexthourvolume', color='blue')
plt.plot(predictions_np, label='Predicted nexthourvolume', color='red')
plt.xlabel('Time')
plt.ylabel('nexthourvolume')
plt.title('Actual vs Predicted nexthourvolume (Neural Network)')
plt.legend()
plt.show()
